![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

#  Zero-Shot Named Entity Recognition in Spark NLP

In this notebook, you will find an example of Zero-Shot NER model (`zero_shot_ner_roberta`) that is the first of its kind and can detect any named entities without using any annotated dataset to train a model. 

`ZeroShotNerModel` annotator also allows extracting entities by crafting appropriate prompts to query **any RoBERTa Question Answering model**. 


You can check the model card here: [Models Hub](https://nlp.johnsnowlabs.com/2022/08/29/zero_shot_ner_roberta_en.html)

In [0]:
import sparknlp
import sparknlp_jsl
import pandas as pd
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.2.4
Spark NLP_JSL Version : 4.2.4
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.2 
 Master 
 spark://10.139.64.5:7077 
 AppName 
 Databricks Shell

## Zero-Shot Clinical NER Pipeline

Now we will create a pipeline for Zero-Shot NER model with only `documentAssembler`, `sentenceDetector`, `tokenizer`, `zero_shot_ner` and `ner_converter` stages. As you can see, we don't use any embeddings model, because it is already included in the model. 

Only the thing that you need to do is create meaningful definitions for the entities that you want to extract. For example; we want to detect `PROBLEM`, `DRUG`, `PATIENT_AGE` and  `ADMISSION_DATE` entities, so we created a dictionary with the questions for detecting these entities and the labels that we want to see in the result. Then we provided this dictionary to the model by using `setEntityDefinitions` parameter.

In [0]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentenceDetector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")
    
zero_shot_ner = ZeroShotNerModel.pretrained("zero_shot_ner_roberta", "en", "clinical/models")\
    .setEntityDefinitions(
        {
            "PROBLEM": ["What is the disease?", "What is his symptom?", "What is her disease?", "What is his disease?", 
                        "What is the problem?" ,"What does a patient suffer", 'What was the reason that the patient is admitted to the clinic?'],
            "DRUG": ["Which drug?", "Which is the drug?", "What is the drug?", "Which drug does he use?", "Which drug does she use?", "Which drug do I use?", "Which drug is prescribed for a symptom?"],
            "ADMISSION_DATE": ["When did patient admitted to a clinic?"],
            "PATIENT_AGE": ["How old is the patient?",'What is the gae of the patient?']
        })\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("zero_shot_ner")\
    .setPredictionThreshold(0.1) # default 0.01

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence", "token", "zero_shot_ner"])\
    .setOutputCol("ner_chunk")\

pipeline = Pipeline(stages = [
    documentAssembler, 
    sentenceDetector, 
    tokenizer, 
    zero_shot_ner, 
    ner_converter])

zero_shot_ner_model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

zero_shot_ner_roberta download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
zero_shot_ner.extractParamMap()

Out[3]: {Param(parent='ZeroShotRobertaNer_5d06c0297d21', name='engine', doc='Deep Learning engine used for this model'): 'tensorflow',
 Param(parent='ZeroShotRobertaNer_5d06c0297d21', name='ignoreEntities', doc='List of entities to ignore'): [],
 Param(parent='ZeroShotRobertaNer_5d06c0297d21', name='predictionThreshold', doc='Minimal confidence score to encode an entity (default is 0.1)'): 0.1,
 Param(parent='ZeroShotRobertaNer_5d06c0297d21', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='ZeroShotRobertaNer_5d06c0297d21', name='batchSize', doc='Size of every batch'): 8,
 Param(parent='ZeroShotRobertaNer_5d06c0297d21', name='maxSentenceLength', doc='Max sentence length to process'): 512,
 Param(parent='ZeroShotRobertaNer_5d06c0297d21', name='caseSensitive', doc='whether to ignore case in tokens for embeddings matching'): True,
 Param(parent='ZeroShotRobertaNer_5d06c0297d21', name='inputCols', doc='previous annotations columns, if renamed'): ['sentence',
 'token'],
 Param(parent='ZeroShotRobertaNer_5d06c0297d21', name='outputCol', doc='output annotation column. can be left default.'): 'zero_shot_ner'}

In [0]:
zero_shot_ner.getClasses()

Out[4]: ['DRUG', 'PATIENT_AGE', 'ADMISSION_DATE', 'PROBLEM']

In [0]:
zero_shot_ner.getPredictionThreshold()

Out[5]: 0.1

In [0]:
text_list = ["The doctor pescribed Majezik for my severe headache.",
             "The patient was admitted to the hospital for his colon cancer.",
             "27 years old patient was admitted to clinic on Sep 1st by Dr. X for a right-sided pleural effusion for thoracentesis."
            ]

data = spark.createDataFrame(text_list, StringType()).toDF("text")

results = zero_shot_ner_model.transform(data)

In [0]:
results.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| zero_shot_ner| ner_chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
The doctor pescri...|[{document, 0, 51...|[{document, 0, 51...|[{token, 0, 2, Th...|[{named_entity, 0...|[{chunk, 21, 27, ...|
The patient was a...|[{document, 0, 61...|[{document, 0, 61...|[{token, 0, 2, Th...|[{named_entity, 0...|[{chunk, 49, 60, ...|
27 years old pati...|[{document, 0, 11...|[{document, 0, 11...|[{token, 0, 1, 27...|[{named_entity, 0...|[{chunk, 0, 11, 2...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

Lets check the NER model results.

In [0]:
results.selectExpr("explode(zero_shot_ner) AS entity")\
       .select(
           "entity.metadata.word",    
           "entity.result",    
           "entity.metadata.sentence",
           "entity.begin",
           "entity.end",
           "entity.metadata.confidence",
           "entity.metadata.question")\
       .show(100, truncate=False)

+-------------+----------------+--------+-----+---+----------+---------------------------------------------------------------+
word |result |sentence|begin|end|confidence|question |
+-------------+----------------+--------+-----+---+----------+---------------------------------------------------------------+
The |O |0 |0 |2 |null |null |
doctor |O |0 |4 |9 |null |null |
pescribed |O |0 |11 |19 |null |null |
Majezik |B-DRUG |0 |21 |27 |0.6467137 |Which drug is prescribed for a symptom? |
for |O |0 |29 |31 |null |null |
my |O |0 |33 |34 |null |null |
severe |B-PROBLEM |0 |36 |41 |0.55263567|What is the problem? |
headache |I-PROBLEM |0 |43 |50 |0.55263567|What is the problem? |
. |O |0 |51 |51 |null |null |
The |O |0 |0 |2 |null |null |
patient |O |0 |4 |10 |null |null |
was |O |0 |12 |14 |null |null |
admitted |O |0 |16 |23 |null |null |
to |O |0 |25 |26 |null |null |
the |O |0 |28 |30 |null |null |
hospital |O |0 |32 |39 |null |null |
for |O |0 |41 |43 |null |null |
his |O |0 |45 |47 |null |null |
colon |B-PROBLEM |0 |49 |53 |0.8898501 |What is his disease? |
cancer |I-PROBLEM |0 |55 |60 |0.8898501 |What is his disease? |
. |O |0 |61 |61 |null |null |
27 |B-PATIENT_AGE |0 |0 |1 |0.6943086 |How old is the patient? |
years |I-PATIENT_AGE |0 |3 |7 |0.6943086 |How old is the patient? |
old |I-PATIENT_AGE |0 |9 |11 |0.6943086 |How old is the patient? |
patient |O |0 |13 |19 |null |null |
was |O |0 |21 |23 |null |null |
admitted |O |0 |25 |32 |null |null |
to |O |0 |34 |35 |null |null |
clinic |O |0 |37 |42 |null |null |
on |O |0 |44 |45 |null |null |
Sep |B-ADMISSION_DATE|0 |47 |49 |0.95646083|When did patient admitted to a clinic? |
1st |I-ADMISSION_DATE|0 |51 |53 |0.95646083|When did patient admitted to a clinic? |
by |O |0 |55 |56 |null |null |
Dr |O |0 |58 |59 |null |null |
. |O |0 |60 |60 |null |null |
X |O |0 |62 |62 |null |null |
for |O |0 |64 |66 |null |null |
a |B-PROBLEM |0 |68 |68 |0.50026655|What was the reason that the patient is admitted to the clinic?|
right-sided |I-PROBLEM |0 |70 |80 |0.50026655|What was the reason that the patient is admitted to the clinic?|
pleural |I-PROBLEM |0 |82 |88 |0.50026655|What was the reason that the patient is admitted to the clinic?|
effusion |I-PROBLEM |0 |90 |97 |0.50026655|What was the reason that the patient is admitted to the clinic?|
for |I-PROBLEM |0 |99 |101|0.50026655|What was the reason that the patient is admitted to the clinic?|
thoracentesis|I-PROBLEM |0 |103 |115|0.50026655|What was the reason that the patient is admitted to the clinic?|
. |O |0 |116 |116|null |null |
+-------------+----------------+--------+-----+---+----------+---------------------------------------------------------------+

In [0]:
results.select(F.explode(F.arrays_zip(results.token.result,
                                      results.zero_shot_ner.result, 
                                      results.zero_shot_ner.metadata,
                                      results.zero_shot_ner.begin, 
                                      results.zero_shot_ner.end)).alias("cols"))\
       .select(F.expr("cols['0']").alias("token"),
               F.expr("cols['1']").alias("ner_label"),
               F.expr("cols['2']['sentence']").alias("sentence"),
               F.expr("cols['3']").alias("begin"),
               F.expr("cols['4']").alias("end"),
               F.expr("cols['2']['confidence']").alias("confidence")).show(50, truncate=100)

+-------------+----------------+--------+-----+---+----------+
 token| ner_label|sentence|begin|end|confidence|
+-------------+----------------+--------+-----+---+----------+
 The| O| 0| 0| 2| null|
 doctor| O| 0| 4| 9| null|
 pescribed| O| 0| 11| 19| null|
 Majezik| B-DRUG| 0| 21| 27| 0.6467137|
 for| O| 0| 29| 31| null|
 my| O| 0| 33| 34| null|
 severe| B-PROBLEM| 0| 36| 41|0.55263567|
 headache| I-PROBLEM| 0| 43| 50|0.55263567|
 .| O| 0| 51| 51| null|
 The| O| 0| 0| 2| null|
 patient| O| 0| 4| 10| null|
 was| O| 0| 12| 14| null|
 admitted| O| 0| 16| 23| null|
 to| O| 0| 25| 26| null|
 the| O| 0| 28| 30| null|
 hospital| O| 0| 32| 39| null|
 for| O| 0| 41| 43| null|
 his| O| 0| 45| 47| null|
 colon| B-PROBLEM| 0| 49| 53| 0.8898501|
 cancer| I-PROBLEM| 0| 55| 60| 0.8898501|
 .| O| 0| 61| 61| null|
 27| B-PATIENT_AGE| 0| 0| 1| 0.6943086|
 years| I-PATIENT_AGE| 0| 3| 7| 0.6943086|
 old| I-PATIENT_AGE| 0| 9| 11| 0.6943086|
 patient| O| 0| 13| 19| null|
 was| O| 0| 21| 23| null|
 admitted| O| 0| 25| 32| null|
 to| O| 0| 34| 35| null|
 clinic| O| 0| 37| 42| null|
 on| O| 0| 44| 45| null|
 Sep|B-ADMISSION_DATE| 0| 47| 49|0.95646083|
 1st|I-ADMISSION_DATE| 0| 51| 53|0.95646083|
 by| O| 0| 55| 56| null|
 Dr| O| 0| 58| 59| null|
 .| O| 0| 60| 60| null|
 X| O| 0| 62| 62| null|
 for| O| 0| 64| 66| null|
 a| B-PROBLEM| 0| 68| 68|0.50026655|
 right-sided| I-PROBLEM| 0| 70| 80|0.50026655|
 pleural| I-PROBLEM| 0| 82| 88|0.50026655|
 effusion| I-PROBLEM| 0| 90| 97|0.50026655|
 for| I-PROBLEM| 0| 99|101|0.50026655|
thoracentesis| I-PROBLEM| 0| 103|115|0.50026655|
 .| O| 0| 116|116| null|
+-------------+----------------+--------+-----+---+----------+

Now we will check the NER chunks.

In [0]:
results.selectExpr("explode(ner_chunk)").show(100, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
col |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
{chunk, 21, 27, Majezik, {chunk -> 0, confidence -> 0.6467137, ner_source -> ner_chunk, entity -> DRUG, sentence -> 0}, []} |
{chunk, 36, 50, severe headache, {chunk -> 1, confidence -> 0.55263567, ner_source -> ner_chunk, entity -> PROBLEM, sentence -> 0}, []} |
{chunk, 49, 60, colon cancer, {chunk -> 0, confidence -> 0.8898501, ner_source -> ner_chunk, entity -> PROBLEM, sentence -> 0}, []} |
{chunk, 0, 11, 27 years old, {chunk -> 0, confidence -> 0.6943086, ner_source -> ner_chunk, entity -> PATIENT_AGE, sentence -> 0}, []} |
{chunk, 47, 53, Sep 1st, {chunk -> 1, confidence -> 0.95646083, ner_source -> ner_chunk, entity -> ADMISSION_DATE, sentence -> 0}, []} |
{chunk, 68, 115, a right-sided pleural effusion for thoracentesis, {chunk -> 2, confidence -> 0.50026655, ner_source -> ner_chunk, entity -> PROBLEM, sentence -> 0}, []}|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

In [0]:
results.select(F.explode(F.arrays_zip(results.ner_chunk.result,
                                      results.ner_chunk.metadata)).alias("cols"))\
       .select(F.expr("cols['0']").alias("chunk"),
               F.expr("cols['1']['entity']").alias("ner_label"),
               F.expr("cols['1']['confidence']").alias("confidence")).show(50, truncate=100)

+------------------------------------------------+--------------+----------+
 chunk| ner_label|confidence|
+------------------------------------------------+--------------+----------+
 Majezik| DRUG| 0.6467137|
 severe headache| PROBLEM|0.55263567|
 colon cancer| PROBLEM| 0.8898501|
 27 years old| PATIENT_AGE| 0.6943086|
 Sep 1st|ADMISSION_DATE|0.95646083|
a right-sided pleural effusion for thoracentesis| PROBLEM|0.50026655|
+------------------------------------------------+--------------+----------+

## LightPipelines

In [0]:
# fullAnnotate in LightPipeline
print (text_list[-1], "\n")

light_model = LightPipeline(zero_shot_ner_model)
light_result = light_model.fullAnnotate(text_list[-1])

chunks = []
entities = []
sentence= []
begin = []
end = []

for n in light_result[0]['ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    sentence.append(n.metadata['sentence'])
    
    

df_clinical = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 
                   'sentence_id':sentence, 'entities':entities})

df_clinical.head(20)

27 years old patient was admitted to clinic on Sep 1st by Dr. X for a right-sided pleural effusion for thoracentesis. 

Out[12]:

,chunks,begin,end,sentence_id,entities
0,27 years old,0,11,0,PATIENT_AGE
1,Sep 1st,47,53,0,ADMISSION_DATE
2,a right-sided pleural effusion for thoracentesis,68,115,0,PROBLEM


In [0]:
light_result[0]

Out[13]: {'zero_shot_ner': [Annotation(named_entity, 0, 1, B-PATIENT_AGE, {'sentence': '0', 'word': '27', 'confidence': '0.6943086', 'question': 'How old is the patient?'}),
 Annotation(named_entity, 3, 7, I-PATIENT_AGE, {'sentence': '0', 'word': 'years', 'confidence': '0.6943086', 'question': 'How old is the patient?'}),
 Annotation(named_entity, 9, 11, I-PATIENT_AGE, {'sentence': '0', 'word': 'old', 'confidence': '0.6943086', 'question': 'How old is the patient?'}),
 Annotation(named_entity, 13, 19, O, {'sentence': '0', 'word': 'patient'}),
 Annotation(named_entity, 21, 23, O, {'sentence': '0', 'word': 'was'}),
 Annotation(named_entity, 25, 32, O, {'sentence': '0', 'word': 'admitted'}),
 Annotation(named_entity, 34, 35, O, {'sentence': '0', 'word': 'to'}),
 Annotation(named_entity, 37, 42, O, {'sentence': '0', 'word': 'clinic'}),
 Annotation(named_entity, 44, 45, O, {'sentence': '0', 'word': 'on'}),
 Annotation(named_entity, 47, 49, B-ADMISSION_DATE, {'sentence': '0', 'word': 'Sep', 'confidence': '0.95646083', 'question': 'When did patient admitted to a clinic?'}),
 Annotation(named_entity, 51, 53, I-ADMISSION_DATE, {'sentence': '0', 'word': '1st', 'confidence': '0.95646083', 'question': 'When did patient admitted to a clinic?'}),
 Annotation(named_entity, 55, 56, O, {'sentence': '0', 'word': 'by'}),
 Annotation(named_entity, 58, 59, O, {'sentence': '0', 'word': 'Dr'}),
 Annotation(named_entity, 60, 60, O, {'sentence': '0', 'word': '.'}),
 Annotation(named_entity, 62, 62, O, {'sentence': '0', 'word': 'X'}),
 Annotation(named_entity, 64, 66, O, {'sentence': '0', 'word': 'for'}),
 Annotation(named_entity, 68, 68, B-PROBLEM, {'sentence': '0', 'word': 'a', 'confidence': '0.50026655', 'question': 'What was the reason that the patient is admitted to the clinic?'}),
 Annotation(named_entity, 70, 80, I-PROBLEM, {'sentence': '0', 'word': 'right-sided', 'confidence': '0.50026655', 'question': 'What was the reason that the patient is admitted to the clinic?'}),
 Annotation(named_entity, 82, 88, I-PROBLEM, {'sentence': '0', 'word': 'pleural', 'confidence': '0.50026655', 'question': 'What was the reason that the patient is admitted to the clinic?'}),
 Annotation(named_entity, 90, 97, I-PROBLEM, {'sentence': '0', 'word': 'effusion', 'confidence': '0.50026655', 'question': 'What was the reason that the patient is admitted to the clinic?'}),
 Annotation(named_entity, 99, 101, I-PROBLEM, {'sentence': '0', 'word': 'for', 'confidence': '0.50026655', 'question': 'What was the reason that the patient is admitted to the clinic?'}),
 Annotation(named_entity, 103, 115, I-PROBLEM, {'sentence': '0', 'word': 'thoracentesis', 'confidence': '0.50026655', 'question': 'What was the reason that the patient is admitted to the clinic?'}),
 Annotation(named_entity, 116, 116, O, {'sentence': '0', 'word': '.'})],
 'document': [Annotation(document, 0, 116, 27 years old patient was admitted to clinic on Sep 1st by Dr. X for a right-sided pleural effusion for thoracentesis., {})],
 'ner_chunk': [Annotation(chunk, 0, 11, 27 years old, {'chunk': '0', 'confidence': '0.6943086', 'ner_source': 'ner_chunk', 'entity': 'PATIENT_AGE', 'sentence': '0'}),
 Annotation(chunk, 47, 53, Sep 1st, {'chunk': '1', 'confidence': '0.95646083', 'ner_source': 'ner_chunk', 'entity': 'ADMISSION_DATE', 'sentence': '0'}),
 Annotation(chunk, 68, 115, a right-sided pleural effusion for thoracentesis, {'chunk': '2', 'confidence': '0.50026655', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'sentence': '0'})],
 'token': [Annotation(token, 0, 1, 27, {'sentence': '0'}),
 Annotation(token, 3, 7, years, {'sentence': '0'}),
 Annotation(token, 9, 11, old, {'sentence': '0'}),
 Annotation(token, 13, 19, patient, {'sentence': '0'}),
 Annotation(token, 21, 23, was, {'sentence': '0'}),
 Annotation(token, 25, 32, admitted, {'sentence': '0'}),
 Annotation(token, 34, 35, to, {'sentence': '0'}),
 Annotation(token, 37, 42, clinic, {'sentence': '0'}),
 Annotation(token, 44, 45, on, {'sentence': '0'}),
 Annotati

### NER Visualizer

For saving the visualization result as html, provide `save_path` parameter in the display function.

In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

for i in text_list:

    light_result = light_model.fullAnnotate(i)
    ner_vis = visualiser.display(light_result[0], label_col='ner_chunk', document_col='document', return_html=True)

    # Change color of an entity label
    # visualiser.set_label_colors({'PROBLEM':'#008080', 'DRUG':'#800080', 'PATIENT_AGE':'#808080'})
    # ner_vis = visualiser.display(light_result[0], label_col='ner_chunk', return_html=True)


    # Set label filter
    # ner_vis = visualiser.display(light_result[0], label_col='ner_chunk', document_col='document',labels=['PROBLEM'], return_html=True)
    
    displayHTML(ner_vis)

The doctor pescribed Majezik DRUG for my severe headache PROBLEM .

The patient was admitted to the hospital for his colon cancer PROBLEM .

27 years old PATIENT_AGE patient was admitted to clinic on Sep 1st ADMISSION_DATE by Dr. X for a right-sided pleural effusion for thoracentesis PROBLEM .

# Save the Model and Load from Disc

Now we will save the Zero-Shot NER model and then we will be able to use this model without definitions. So our model will have the same labels that we defined before.

In [0]:
# save model

zero_shot_ner.write().overwrite().save("dbfs:/databricks/driver/zero_shot_ner_model")

In [0]:
# load from disc and create a new pipeline

zero_shot_ner_local = ZeroShotNerModel.load("dbfs:/databricks/driver/zero_shot_ner_model")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("zero_shot_ner")

ner_converter_local = sparknlp.annotators.NerConverter()\
    .setInputCols(["sentence", "token", "zero_shot_ner"])\
    .setOutputCol("ner_chunk")\

pipeline_local = Pipeline(stages = [
    documentAssembler, 
    sentenceDetector, 
    tokenizer, 
    zero_shot_ner_local, 
    ner_converter_local])

zero_shot_ner_model_local = pipeline_local.fit(spark.createDataFrame([[""]]).toDF("text"))

In [0]:
zero_shot_ner_local.getClasses()

Out[17]: ['DRUG', 'PATIENT_AGE', 'ADMISSION_DATE', 'PROBLEM']

In [0]:
# check the results

local_results = zero_shot_ner_model_local.transform(data)

local_results.select(F.explode(F.arrays_zip(local_results.ner_chunk.result,
                                            local_results.ner_chunk.metadata)).alias("cols"))\
             .select(F.expr("cols['0']").alias("chunk"),
                     F.expr("cols['1']['entity']").alias("ner_label"),
                     F.expr("cols['1']['confidence']").alias("confidence")).show(50, truncate=100)

+------------------------------------------------+--------------+----------+
 chunk| ner_label|confidence|
+------------------------------------------------+--------------+----------+
 Majezik| DRUG| 0.6467137|
 severe headache| PROBLEM|0.55263567|
 colon cancer| PROBLEM| 0.8898501|
 27 years old| PATIENT_AGE| 0.6943086|
 Sep 1st|ADMISSION_DATE|0.95646083|
a right-sided pleural effusion for thoracentesis| PROBLEM|0.50026655|
+------------------------------------------------+--------------+----------+

# NER Question Generator

`NerQuestionGenerator` annotator helps you build questions on the fly using 2 entities from different labels (preferably a subject and a verb). For example, let's suppose you have an NER model, able to detect `PATIENT`and `ADMISSION` in the following text:

`John Smith was admitted Sep 3rd to Mayo Clinic`
- PATIENT: `John Smith`
- ADMISSION: `was admitted`

You can add the following annotator to construct questions using PATIENT and ADMISSION:

```python
# setEntities1 says which entity from NER goes first in the question
# setEntities2 says which entity from NER goes second in the question
# setQuestionMark to True adds a '?' at the end of the sentence (after entity 2)
# To sum up, the pattern is     [QUESTIONPRONOUN] [ENTITY1] [ENTITY2] [QUESTIONMARK]

qagenerator = NerQuestionGenerator()\
  .setInputCols(["ner_chunk"])\
  .setOutputCol("question")\
  .setQuestionMark(True)\
  .setQuestionPronoun("When")\
  .setStrategyType("Paired")\
  .setEntities1(["PATIENT"])\
  .setEntities2(["ADMISSION"])
```
In the column `question` you will find: `When John Smith was admitted?`. Likewise you could have `Where` or any other question pronoun you may need.

You can use those questions in a QuestionAnsweringModel or ZeroShotNER (any model which requires a question as an input. Let's see the case of QA.

```python
qa = BertForQuestionAnswering.pretrained("bert_qa_spanbert_finetuned_squadv1","en") \
  .setInputCols(["question", "document"]) \
  .setOutputCol("answer") \
  .setCaseSensitive(True)
```
The result will be:

```bash
+--------------------------------------------------------+-----------------------------+
|question                                                |answer                       |
+--------------------------------------------------------+-----------------------------+
|[{document, 0, 25, When John Smith was admitted ? ...}] |[{chunk, 0, 8, Sep 3rd ...}] |
+--------------------------------------------------------+-----------------------------+
```
Strategies:
- Paired: First chunk of Entity 1 will be grouped with first chunk of Entity 2, second with second, third with third, etc (one-vs-one)
- Combined: A more flexible strategy to be used in case the number of chukns in Entity 1 is not aligned with the number of chunks in Entityt 2. The first chunk from Entity 1 will be grouped with all chunks in Entity 2, the second chunk in Entity 1 with again be grouped with all the chunks in Entity 2, etc (one-vs-all).